In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -qq /content/drive/'My Drive'/Code/tomatoes.zip

In [3]:
import cv2
import os
from tqdm.notebook import tqdm
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from fastai import *
from fastai.vision import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.applications import MobileNetV2

In [4]:
dataset_path = '/content/PlantVillage'

class_name = []
images = []
labels = []
for index, folder in enumerate(os.listdir(dataset_path)):
  class_name.append(folder)
  image_path = os.path.join(dataset_path, folder)
  # print(image_path, len(os.listdir(image_path)))
  for image in os.listdir(image_path):
    labels.append(index)
    images.append(os.path.join(dataset_path, folder, image))

X = []
for index, path in enumerate(tqdm(images)):
  try:
    X.append(cv2.resize(cv2.imread(path),dsize=(224,224)))
  except:
    labels.pop(index)
X = np.array(X)
y = np.array(labels)
Y = to_categorical(y, num_classes=len(class_name))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4,stratify = Y, shuffle=True, random_state=100)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify = y_test, shuffle=True, random_state=100)
print(X_train.shape, X_test.shape,X_val.shape)
print(y_train.shape, y_test.shape,y_val.shape)

(7426, 224, 224, 3) (2476, 224, 224, 3) (2476, 224, 224, 3)
(7426, 5) (2476, 5) (2476, 5)


In [ ]:
def get_model():
    mobilenet = MobileNetV2(
        weights='imagenet',
        include_top=False)
    # Dong bang cac layer
    #for layer in mobilenet.layers:
    #   layer.trainable = False

    input = Input(shape=(256, 256, 3))
    x = Conv2D(3, (3, 3), padding='same')(input)
    x = mobilenet(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(5,activation = 'softmax', name='root')(x)

    # model
    model = Model(input,output)
    
    optimizer = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()

    return model

model = get_model()

filepath="/content/weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=True,
    brightness_range=[0.2,1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1./255)

history=model.fit(aug.flow(X_train, y_train, batch_size=32),
                               epochs=25, steps_per_epoch=len(X_train)//32,
                               validation_data=aug.flow(X_val,y_val,
                               batch_size=64), callbacks=callbacks_list)


9412608/9406464 [==============================] - 0s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 3)       84        
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, None, None, 1280)  2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280)              5120      
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________

In [ ]:
from keras.models import load_model
from sklearn.metrics import classification_report

for i in range(1,10):
  modellink = '/content/weights-' + '0' + str(i) + '.hdf5'
  model = load_model(modellink)
  #y_pred = model.predict(X_test, batch_size=64, verbose=1)
  #y_pred_bool = np.argmax(y_pred, axis=1)
  #print(classification_report(y_test, y_pred_bool))
  scores = model.evaluate(X_test, y_test)
  print(f"Test Accuracy: {scores[1]*100}")

for i in range(10,26):
  modellink = '/content/weights-' + str(i) + '.hdf5' 
  model = load_model(modellink)
  scores = model.evaluate(X_test, y_test)
  print(f"Test Accuracy: {scores[1]*100}")